# Functores Aplicativos

Tenemos el siguiente problema. No podemos expresar operaciones entre 2 instancias de un functor:

```haskell
aplicaEntreFunctores (+)  (Just 5)     (Just 11)
aplicaEntreFunctores (++) [1,2,3]      [4,5,6]
aplicaEntreFunctores (&&) (Maybe True) (Maybe False)
```

Lo que sí podemos hacer, es mapear un functor a un functor que contiene una función:

In [ ]:
:t fmap (+) (Just 5) -- Just ((+) 5) = Just (5 +)

Lo anterior, lo podríamos lograr si tuvieramos alguna de las siguientes 2 opciones:

- `apply :: f (a -> b) -> f a -> f b`
- `lift  :: (a -> b -> c) -> f a -> f b -> f c`

Además, necesitaríamos lo siguiente para crear functores:

- `pure :: a -> f a`

En esencia, lo que queremos lograr es esto:

![Ilustración de Aplicativos](applicative.png)

**¿Por qué no modificamos los functores para admitir lo anterior?**: No todos los functores pueden definir una función `pure`.

Puesto que los functores no van a cumplir nuestras demandas anteriores, creamos una nueva clase llamada `Aplicativo`. Sus funciones serán `pure`, `<*>` (que corresponde a `apply` y `liftA2` que corresponde a `lift`.

_Nota: `<*>` debe definirse en función de `liftA2` y viceversa._

_Nota 2: `<*>` es infijo asociativo a la izquierda, con jerarquía 4._

## Leyes

- Identidad: `pure id <*> x = x`.
- Composición: `pure (.) <*> x <*> y <*> z = x <*> (y <*> z)`.
- Homomorfismo: `pure f <*> pure x = pure (f x)`.
- Intercambio: `u <*> pure y = pure ($ y) <*> u`.

## Instancias

Definamos a las listas como instancias de aplicativos.

Definamos a `Maybe` como una instancia de aplicativo.

Definamos a las funciones como instancias de aplicativos.

En los Aplicativos, las instancias para los tipos dejan de ser necesariamente únicas. Veamos el ejemplo con listas:

In [ ]:
(++) <$> ["Hola", "Lambda"] <*> ["Mundo", "Club"]

En la definición por defecto, nos generar todas las combinaciones de 2 listas bajo una función. Pero también sería un aplicativo válido el que resulta en lo siguiente:

```haskell
> (++) <$> ["Hola", "Lambda"] <*> ["Mundo", "Club"]
> ["HolaMundo", "LambdaClub"]
```

Define la segunda instancia de aplicativos para listas con el comportamiento anterior. Utiliza la función `zipWith`.

In [ ]:
:t zipWith

## Casos de uso

- Punto medio entre functores y monoides.
- Parsear.

In [ ]:
import Data.Char (isDigit, isLetter, isSpace)
import Text.Read (readMaybe)

data Usuario = Usuario { idUsuario :: Int
                       , edad      :: Int
                       , nombre    :: String
                       } deriving Show

parseaID :: String -> Maybe Int
parseaID = readMaybe . takeWhile isDigit

parseaEdad :: String -> Maybe Int
parseaEdad text = let (_:xs) = dropWhile (not . isSpace) text
                    in readMaybe $ takeWhile isDigit xs

parseaNombre :: String -> Maybe String
parseaNombre = Just . takeWhile isLetter . dropWhile (not . isLetter)

In [ ]:
parseaID     "9 20 jpyamamoto"
parseaEdad   "9 20 jpyamamoto"
parseaNombre "9 20 jpyamamoto"

In [ ]:
parseaUsuario :: String -> Maybe Usuario
parseaUsuario u = Usuario <$> parseaID u <*> parseaEdad u <*> parseaNombre u

In [ ]:
parseaUsuario "1 20 jpyamamoto"
parseaUsuario "2 92 alonzochurch"
parseaUsuario "3 alanturing 41"

fmap parseaUsuario ["1 20 jpyamamoto", "2 92 alonzochurch", "3 alanturing 41"]

Concurrencia:

```haskell
#!/usr/bin/env stack
-- stack script --resolver lts-12.21
import Conduit
import UnliftIO

main :: IO ()
main = do
  write2Files
  runConduitRes $
    (sourceFile "file1.txt" *> sourceFile "file2.txt") .|
    sink

write2Files = runConcurrently $
      Concurrently (writeFile "file1.txt" "this is file 1")
   *> Concurrently (writeFile "file2.txt" "this is file 2")

sink = getZipSink $
      ZipSink (sinkFile "output1.txt")
   *> ZipSink (sinkFile "output2.txt")
```

In [ ]:
:t (*>)